
# Лабораторная работа №2  
## Анализ ассоциативных правил (Apriori и FPGrowth) — датасет *TV_Shows*

**Цель работы:** исследовать методы анализа ассоциативных правил на датасете, содержащем информацию о совместно просматриваемых телешоу, с использованием алгоритмов **Apriori** и **FPGrowth**.

В работе выполняются следующие шаги:

1. Загрузка и описание датасета (транзакции: одновременно просмотренные или связанные телешоу).
2. Первичный анализ: распределение длин транзакций, список уникальных шоу и др.
3. Преобразование данных в бинарный формат (one-hot encoding).
4. Поиск частых наборов элементов с помощью алгоритмов **Apriori** и **FPGrowth** при начальных параметрах:  
   - минимальная поддержка `min_support = 0.02`  
   - минимальная достоверность (confidence) `min_confidence = 0.3`
5. Генерация ассоциативных правил, расчёт метрик (**support**, **confidence**, **lift**, **conviction**).
6. Выделение полезных и тривиальных правил, анализ лифта и достоверности.
7. Исследование влияния параметров `min_support` и `min_confidence` на количество и качество правил.
8. Алгоритмическое определение минимальных значений поддержки для наборов из 1, 2, ... объектов.
9. Визуализация ассоциативных правил:  
   - граф на основе правил;  
   - собственный способ визуализации (scatter-графики метрик, диаграммы лучших правил).
10. Формирование выводов по результатам экспериментов.


## 1. Импорт библиотек и загрузка данных

In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

import networkx as nx

# Настройки отображения
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (8, 5)


In [ ]:


all_data = pd.read_csv('TV_Shows.csv', on_bad_lines='skip')
print("Размерность исходной таблицы:", all_data.shape)
all_data.head()


## 2. Первичный анализ и описание данных

In [ ]:

# Общая информация о данных
all_data.info()


In [ ]:

# Случайный фрагмент данных
all_data.sample(5, random_state=42)


### 2.1. Анализ длин транзакций

In [ ]:



transaction_lengths = all_data.notnull().sum(axis=1)
print("Минимальная длина транзакции:", transaction_lengths.min())
print("Максимальная длина транзакции:", transaction_lengths.max())
print("Средняя длина транзакции:", transaction_lengths.mean())

plt.figure(figsize=(8, 4))
plt.hist(transaction_lengths, bins=range(1, transaction_lengths.max() + 2), edgecolor='black')
plt.xlabel('Длина транзакции (кол-во шоу)')
plt.ylabel('Частота')
plt.title('Распределение длин транзакций')
plt.tight_layout()
plt.show()


### 2.2. Список уникальных телешоу

In [ ]:

# Переводим DataFrame в numpy-массив
np_data_raw = all_data.to_numpy()

transactions = [
    [elem for elem in row[1:] if isinstance(elem, str)]
    for row in np_data_raw
]

# Удаляем пустые транзакции
transactions = [t for t in transactions if len(t) > 0]

print("Пример 5 транзакций:")
for t in transactions[:5]:
    print(t)

# Список уникальных шоу
unique_items = set()
for row in transactions:
    for elem in row:
        unique_items.add(elem)

print("\nКоличество уникальных телешоу:", len(unique_items))
print("Первые 30 шоу:")
sorted(list(unique_items))[:30]


## 3. Преобразование данных в бинарный формат (one-hot encoding)

In [ ]:

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

data = pd.DataFrame(te_ary, columns=te.columns_)
print("Размерность бинарной матрицы (транзакции × шоу):", data.shape)
data.head()


## 4. Алгоритм Apriori: частые наборы и ассоциативные правила

### 4.1. Поиск частых наборов (начальные параметры: min_support = 0.02)

In [ ]:

min_support_init = 0.02
min_confidence_init = 0.3

frequent_ap = apriori(data, min_support=min_support_init, use_colnames=True)
frequent_ap['length'] = frequent_ap['itemsets'].apply(len)

print("Число частых наборов (Apriori):", len(frequent_ap))
frequent_ap.sort_values(['length', 'support'], ascending=[True, False]).head(10)


### 4.2. Генерация ассоциативных правил (Apriori)

In [ ]:

rules_ap = association_rules(
    frequent_ap,
    metric="confidence",
    min_threshold=min_confidence_init
)

print("Число ассоциативных правил (Apriori):", len(rules_ap))

# Добавим длины левой и правой части правила
rules_ap['antecedent_len'] = rules_ap['antecedents'].apply(len)
rules_ap['consequent_len'] = rules_ap['consequents'].apply(len)

# Выведем 10 правил по lift
rules_ap.sort_values('lift', ascending=False).head(10)


### 4.3. Полезные vs тривиальные правила

In [ ]:

# Тривиальные правила: высокий confidence, но lift ~ 1 (нет реального усиления связи)
trivial_rules = rules_ap[(rules_ap['confidence'] >= min_confidence_init) & (rules_ap['lift'].between(0.95, 1.05))]

# Потенциально полезные: и confidence, и lift заметно больше 1
useful_rules = rules_ap[(rules_ap['confidence'] >= min_confidence_init) & (rules_ap['lift'] > 1.2)]

print("Тривиальные правила (пример до 10):")
trivial_rules.sort_values('confidence', ascending=False).head(10)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

print("\nПолезные правила (пример до 10):")
useful_rules.sort_values('lift', ascending=False).head(10)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


## 5. Алгоритм FPGrowth: частые наборы и правила

### 5.1. Частые наборы (FPGrowth, те же параметры)

In [ ]:

frequent_fp = fpgrowth(data, min_support=min_support_init, use_colnames=True)
frequent_fp['length'] = frequent_fp['itemsets'].apply(len)

print("Число частых наборов (FPGrowth):", len(frequent_fp))
frequent_fp.sort_values(['length', 'support'], ascending=[True, False]).head(10)


### 5.2. Правила (FPGrowth)

In [ ]:

rules_fp = association_rules(
    frequent_fp,
    metric="confidence",
    min_threshold=min_confidence_init
)

print("Число ассоциативных правил (FPGrowth):", len(rules_fp))

rules_fp['antecedent_len'] = rules_fp['antecedents'].apply(len)
rules_fp['consequent_len'] = rules_fp['consequents'].apply(len)

rules_fp.sort_values('lift', ascending=False).head(10)


## 6. Минимальные значения поддержки для наборов разной длины

In [ ]:

# Используем результаты Apriori (frequent_ap)
# Для каждого k = 1, 2, ..., найдём минимальное значение поддержки среди частых наборов длины k

min_support_by_len = (
    frequent_ap
    .groupby('length')['support']
    .min()
    .rename('min_support')
    .reset_index()
)

print("Минимальная поддержка среди частых наборов заданной длины (Apriori):")
min_support_by_len



> Интерпретация: для наборов из k шоу значения поддержки не могут быть меньше указанных в таблице.  
> Эти значения можно рассматривать как **оценку минимального порога поддержки**, при котором ещё существуют частые наборы длины k для данного датасета.


## 7. Влияние параметров min_support и min_confidence на правила (Apriori)

In [ ]:

support_values = [0.01, 0.02, 0.03, 0.05]
confidence_values = [0.2, 0.3, 0.4]

results_experiments = []

for sup in support_values:
    freq_tmp = apriori(data, min_support=sup, use_colnames=True)
    if freq_tmp.empty:
        continue
    for conf in confidence_values:
        rules_tmp = association_rules(freq_tmp, metric="confidence", min_threshold=conf)
        if len(rules_tmp) == 0:
            results_experiments.append({
                'min_support': sup,
                'min_confidence': conf,
                'n_rules': 0,
                'mean_confidence': np.nan,
                'mean_lift': np.nan
            })
        else:
            results_experiments.append({
                'min_support': sup,
                'min_confidence': conf,
                'n_rules': len(rules_tmp),
                'mean_confidence': rules_tmp['confidence'].mean(),
                'mean_lift': rules_tmp['lift'].mean()
            })

experiments_df = pd.DataFrame(results_experiments)
experiments_df.sort_values(['min_support', 'min_confidence'])



**Наблюдения (ожидаемые):**

- При **увеличении `min_support`** число частых наборов и ассоциативных правил, как правило, **уменьшается**, так как остаются только самые распространённые комбинации телешоу.
- При **увеличении `min_confidence`** отсекаются правила с низкой достоверностью — общее число правил уменьшается, но средняя достоверность и лифт у оставшихся правил растут.
- Слишком низкие пороги (`min_support`, `min_confidence`) приводят к огромному количеству правил, многие из которых трудно интерпретировать; слишком высокие — к отсутствию или очень малому количеству правил.


## 8. Визуализация ассоциативных правил в виде графа

In [ ]:

# Для визуализации возьмём небольшой набор правил, чтобы граф был читаемым
rules_for_graph = (
    rules_ap
    .sort_values('confidence', ascending=False)
    .head(20)  # можно менять количество
    .copy()
)

# Строковое представление левой и правой части правила
rules_for_graph['antecedent_str'] = rules_for_graph['antecedents'].apply(lambda x: ', '.join(list(x)))
rules_for_graph['consequent_str'] = rules_for_graph['consequents'].apply(lambda x: ', '.join(list(x)))

G = nx.DiGraph()

# Добавляем узлы и рёбра
for _, row in rules_for_graph.iterrows():
    a = row['antecedent_str']
    c = row['consequent_str']
    conf = row['confidence']
    lift = row['lift']
    G.add_node(a)
    G.add_node(c)
    G.add_edge(a, c, confidence=conf, lift=lift)

plt.figure(figsize=(10, 7))
pos = nx.spring_layout(G, k=0.7, seed=42)

nx.draw(
    G, pos,
    with_labels=True,
    node_size=2000,
    node_color='lightblue',
    font_size=8,
    arrows=True,
    arrowstyle='->',
    arrowsize=15
)

# Подписи рёбер – значение confidence
edge_labels = { (u, v): f"{d['confidence']:.2f}" for u, v, d in G.edges(data=True) }
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

plt.title('Граф ассоциативных правил (узлы = наборы шоу, рёбра = правила, подпись = confidence)')
plt.tight_layout()
plt.show()



**Интерпретация графа:**

- **Узлы** соответствуют наборам телешоу (левая или правая части правил).  
- **Направленные рёбра** показывают ассоциативные правила вида *antecedent → consequent*.
- Подпись на ребре — значение **confidence** (достоверности правила).  
- Узлы с большим количеством входящих/исходящих рёбер обозначают телешоу, которые часто встречаются в связке с другими и могут быть «якорными» рекомендациями.


## 9. Собственный способ визуализации ассоциативных правил и метрик

### 9.1. Scatter-график: поддержка vs достоверность, цвет = лифт

In [ ]:

# Для удобства ограничим количество отображаемых правил
rules_plot = rules_ap.copy()
rules_plot = rules_plot.sort_values('support', ascending=False).head(200)

plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    rules_plot['support'],
    rules_plot['confidence'],
    c=rules_plot['lift'],
    cmap='viridis',
    alpha=0.8
)
plt.colorbar(scatter, label='Lift')
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.title('Ассоциативные правила: Support vs Confidence (цвет = Lift)')
plt.tight_layout()
plt.show()


### 9.2. Топ-правила по лифту в виде горизонтальной диаграммы

In [ ]:

top_lift = (
    rules_ap
    .copy()
    .sort_values('lift', ascending=False)
    .head(15)
)

top_lift['rule_str'] = top_lift.apply(
    lambda row: f"{', '.join(list(row['antecedents']))} -> {', '.join(list(row['consequents']))}",
    axis=1
)

plt.figure(figsize=(10, 6))
sns.barplot(
    x='lift',
    y='rule_str',
    data=top_lift,
    orient='h'
)
plt.xlabel('Lift')
plt.ylabel('Правило')
plt.title('Топ-15 правил по лифту')
plt.tight_layout()
plt.show()



**Пояснение к визуализациям:**

1. **Scatter-график Support vs Confidence (цвет = Lift)**  
   Позволяет одновременно анализировать частоту сочетания телешоу (support), силу правила (confidence) и «усиление» связи (lift).  
   Можно выделить:
   - редкие, но сильные комбинации (низкий support, высокий lift);
   - массовые и сильные (высокий support и lift);
   - тривиальные (lift ≈ 1).

2. **Горизонтальный barplot топ-правил по лифту**  
   Явно показывает наиболее интересные ассоциативные правила в виде читаемых строк вида  
   `шоу(а) -> шоу(а)` c максимальным лифтом.


## 10. Выводы по лабораторной работе (датасет TV_Shows)


В ходе выполнения лабораторной работы на датасете **TV_Shows** были получены следующие результаты:

1. Проведён анализ транзакций (совместно встречающихся телешоу):  
   - построено распределение длин транзакций;  
   - получен список уникальных телешоу и их количество.

2. Данные преобразованы в бинарный формат (one-hot encoding), что позволило применить алгоритмы поиска частых наборов элементов.

3. С использованием алгоритма **Apriori** при параметрах `min_support = 0.02` и `min_confidence = 0.3` найдены частые наборы шоу и ассоциативные правила.  
   Анализ метрик **support**, **confidence**, **lift** и **conviction** позволил выделить как тривиальные, так и потенциально полезные правила.

4. Алгоритм **FPGrowth** применён для того же датасета и тех же параметров. Он показал аналогичные результаты, при этом являясь более эффективным при увеличении размера данных.

5. По результатам Apriori вычислены минимальные значения поддержки для наборов различной длины (1, 2, ... шоу), что позволяет оценить разумные нижние пороги `min_support` для поиска правил заданной сложности.

6. Исследовано влияние параметров `min_support` и `min_confidence` на количество и качество правил:  
   - увеличение порогов приводит к уменьшению числа правил и росту средней достоверности и лифта;  
   - слишком низкие пороги дают много, но не всегда содержательных правил.

7. Построен граф ассоциативных правил, наглядно показывающий связи между наборами телешоу. На его основе можно выделить «центральные» шоу и кластеры взаимосвязанных шоу, что важно, например, для систем рекомендаций.

8. Реализованы дополнительные визуализации (scatter-график метрик и barplot топ-правил), которые упрощают содержательный анализ правил и выбор наиболее интересных закономерностей.

